#Projeto 1 de Visualização Computacional (SCC0252)
### Diogo Castanho Emídio – 11297274
### Eduardo Souza Rocha – 11218692
### Nathan Rodrigues de Oliveira – 11218938

#Introdução

Para a realização do presente projeto, o *dataset* escolhido foi o referente a dados de vencedores do Prêmio Nobel. O objetivo, por sua vez, é explorar esses dados, criando uma solução através do pré-processamento e de técnicas de vizualização. Tais dados foram extraídos de um arquivo .csv, disponível no [site](https://www.kaggle.com/datasets/bahramjannesarr/nobel-prize-from-1901-till-2020) recomendado na descrição do projeto e carregado no [GitHub](https://github.com/NathanTBP/nobeldatavisualization).

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Carrega o dataset remotamente
url = "https://raw.githubusercontent.com/NathanTBP/nobeldatavisualization/main/nobel_final.csv"
df = pd.read_csv(url)

# Carrega o dataset localmente
# df = pd.read_csv("nobel_final.csv")

# Visualiza do começo dos dados
df.head(5)

,firstname,surname,born_country_code,died_country_code,gender,year,category,share,name_of_university,city_of_university,country_of_university,born_month,age,age_get_prize
0,Wilhelm Conrad,Röntgen,DE,DE,male,1901,physics,1,Munich University,Munich,Germany,Mar,78,56
1,Hendrik A.,Lorentz,NL,NL,male,1902,physics,2,Leiden University,Leiden,the Netherlands,Jul,75,49
2,Pieter,Zeeman,NL,NL,male,1902,physics,2,Amsterdam University,Amsterdam,the Netherlands,May,78,37
3,Henri,Becquerel,FR,FR,male,1903,physics,2,École Polytechnique,Paris,France,Dec,56,51
4,Pierre,Curie,FR,FR,male,1903,physics,4,École municipale de physique et de chimie indu...,Paris,France,May,47,44


In [ ]:
# Visualiza o fim dos dados
df.tail(5)

,firstname,surname,born_country_code,died_country_code,gender,year,category,share,name_of_university,city_of_university,country_of_university,born_month,age,age_get_prize
918,Peter,Handke,AT,NaN,male,2019,literature,1,NaN,NaN,NaN,Dec,78,77
919,Abiy,Ahmed Ali,ET,NaN,male,2019,peace,1,NaN,NaN,NaN,Aug,44,43
920,Abhijit,Banerjee,IN,NaN,male,2019,economics,3,Massachusetts Institute of Technology (MIT),Cambridge MA,USA,Feb,59,58
921,Esther,Duflo,FR,NaN,female,2019,economics,3,Massachusetts Institute of Technology (MIT),Cambridge MA,USA,Oct,48,47
922,Michael,Kremer,US,NaN,male,2019,economics,3,Harvard University,Cambridge MA,USA,Nov,56,55


Em uma primeira visão do conjunto de dados, é possivel perceber sobre o que se trata: documentar quem ganhou o Prêmio Nobel de 1901 a 2019. Contendo um total de 923 entradas, possui como atributos:
- Nome e sobrenome (`firstname` e `surname`, respectivamente, ambos *strings*);
- Sigla do país de nascimento e de morte (`born_country_code` e `died_country_code`, respectivamente, ambos *strings* de duas letras maiúsculas);
- Gênero (`gender`, *string* que assume apenas os valores *male* ou *female*);
- Categoria (`category`, *string* que assume apenas os valores *medicine*, *physics*, *chemistry*, *literature*, *peace* ou *economics*);
- Quantas pessoas dividiram o prêmio (`share`, inteiro de 1 a 4);
- Nome, cidade e país da universidade (`name_of_university`, `city_of_university` e `country_of_university`, respectivamente, todos *strings*);
- Mês de nascimento (`born_month`, *string* de três letras indicando o mês em inglês);
- Idade em 2020 ou de falecimento (`age`, inteiro);
- Idade em que ganhou o prêmio (`age_get_prize`, inteiro).

#Pré-processamento

Antes de qualquer trabalho com dados, é necessário realizar um pré-processamento, a fim de estudá-los (discriminar numéricos de nominais) e tratar valores espúrios, anômalos ou faltantes (oriundos de erros de medida/coleta ou *outliers*). Dito isso, é interessante contar a quantidade de cada valor das colunas categóricas de menor variedade (gênero e categoria):

In [ ]:
# Imprime o tamnho do dataset e a quantidade de cada gênero e categoria
print("Tamanho do dataset:", len(df))
print("\nGêneros:")
print(df["gender"].value_counts())
print("\nCategorias:")
print(df["category"].value_counts())

Tamanho do dataset: 923

Gêneros:
male      869
female     54
Name: gender, dtype: int64

Categorias:
medicine      219
physics       213
chemistry     184
literature    116
peace         107
economics      84
Name: category, dtype: int64


Em seguida, deve-se mapear valores não válidos (no caso, NaN) por atributo:

In [ ]:
# Imprime a quantidade de valores NaN em cada atributo
def count_nan(df):
    print("Quantidade de valores NaN em:\n")
    for column in df.columns:
        print(column + ':', df[column].isna().sum())

count_nan(df)

Quantidade de valores NaN em:

firstname: 0
surname: 2
born_country_code: 0
died_country_code: 306
gender: 0
year: 0
category: 0
share: 0
name_of_university: 226
city_of_university: 231
country_of_university: 230
born_month: 0
age: 0
age_get_prize: 0


##Redução de dimensionalidade

Como é possível observar, cerca de um terço das instâncias não possui o país em que o vencedor morreu. Isso pode se dar tanto por erro na inserção dos dados quanto por esses vencedores ainda estarem vivos (pelo menos na época em que o *dataset* foi populado com suas informações). Além disso, alguns atributos não possuem tanta relevância para a visualização, como mês de nascimento e idade em 2020 ou de falecimento. Assim, para simplificar o *dataset* e reduzir a dimensionalidade, tais atributos foram removidos:

In [ ]:
# Remove alguns atributos
df.drop(columns=["died_country_code", "born_month", "age"], inplace=True)
count_nan(df)

Quantidade de valores NaN em:

firstname: 0
surname: 2
born_country_code: 0
gender: 0
year: 0
category: 0
share: 0
name_of_university: 226
city_of_university: 231
country_of_university: 230
age_get_prize: 0


###Nome e sobrenome

Ainda na redução de dimensionalidade, uma ação que pode ser feita é a concatenação de nome e sobrenome. Essa operação não somente simplificaria o *dataset* como resolveria o problema de sobrenome NaN:

In [ ]:
# Imprime as instâncias com sobrenome NaN
df[df["surname"].isna()]

,firstname,surname,born_country_code,gender,year,category,share,name_of_university,city_of_university,country_of_university,age_get_prize
515,Le Duc Tho,NaN,VN,male,1973,peace,2,NaN,NaN,NaN,62
533,Aung San Suu Kyi,NaN,MM,female,1991,peace,1,NaN,NaN,NaN,46


Para isso, fez-se a junção de ambos os atributos, considerando aqueles dois valores não válidos:

In [ ]:
# Concatena nome e sobrenome
df["firstname"] += ' ' + df["surname"].fillna('')
df.drop(columns="surname", inplace=True)
df.rename(columns={"firstname": "name"}, inplace=True)
df.head(5)

,name,born_country_code,gender,year,category,share,name_of_university,city_of_university,country_of_university,age_get_prize
0,Wilhelm Conrad Röntgen,DE,male,1901,physics,1,Munich University,Munich,Germany,56
1,Hendrik A. Lorentz,NL,male,1902,physics,2,Leiden University,Leiden,the Netherlands,49
2,Pieter Zeeman,NL,male,1902,physics,2,Amsterdam University,Amsterdam,the Netherlands,37
3,Henri Becquerel,FR,male,1903,physics,2,École Polytechnique,Paris,France,51
4,Pierre Curie,FR,male,1903,physics,4,École municipale de physique et de chimie indu...,Paris,France,44


In [ ]:
# Imprime a quantidade de valores NaN em cada atributo
count_nan(df)

Quantidade de valores NaN em:

name: 0
born_country_code: 0
gender: 0
year: 0
category: 0
share: 0
name_of_university: 226
city_of_university: 231
country_of_university: 230
age_get_prize: 0


##Tratamento de valores NaN nas universidades

Muitos dados referentes às universidades estão também ausentes. Entretanto, é preciso analisar por que há inconsistência entre as quantidades dos três atributos relacionados:

In [ ]:
# Imprime as instâncias com nome de universidade mas não cidade ou país
df[df["name_of_university"].notna() & (df["city_of_university"].isna() | df["country_of_university"].isna())]

,name,born_country_code,gender,year,category,share,name_of_university,city_of_university,country_of_university,age_get_prize
809,Richard F. Heck,US,male,2010,chemistry,3,University of Delaware,NaN,USA,79
834,Robert J. Lefkowitz,US,male,2012,chemistry,2,Howard Hughes Medical Institute,NaN,NaN,69
888,Rainer Weiss,DE,male,2017,physics,2,LIGO/VIRGO Collaboration,NaN,NaN,85
889,Barry C. Barish,US,male,2017,physics,4,LIGO/VIRGO Collaboration,NaN,NaN,81
890,Kip S. Thorne,US,male,2017,physics,4,LIGO/VIRGO Collaboration,NaN,NaN,77


Logo, conclui-se que todas que não possuem nome também não têm cidade nem país (o que é lógico, uma vez que é possível ganhar o prêmio sem vínculo com alguma universidade), mas o contrário não é verdadeiro. Isso provavelmente é oriundo de algum erro na inserção e deverá ser tratado.

###Relação entre nome e país

Entre as inconsistências, há o caso em que as universidades possuem nome mas não cidade nem país, ocorrendo isso com duas instituições. Primeiro foi tratado o Howard Hughes Medical Institute:

In [ ]:
# Imprime as instâncias com a universidade em questão
df[df["name_of_university"] == "Howard Hughes Medical Institute"]

,name,born_country_code,gender,year,category,share,name_of_university,city_of_university,country_of_university,age_get_prize
834,Robert J. Lefkowitz,US,male,2012,chemistry,2,Howard Hughes Medical Institute,NaN,NaN,69
870,Paul Modrich,US,male,2015,chemistry,3,Howard Hughes Medical Institute,Durham NC,USA,69


Já que existe outra instância com a mesma universidade e ainda contendo os campos desejados, é possível atribuir os mesmos valores:

In [ ]:
# Copia os valores
city    = df[(df["name_of_university"] == "Howard Hughes Medical Institute") & df["city_of_university"].notna()]["city_of_university"].item()
country = df[(df["name_of_university"] == "Howard Hughes Medical Institute") & df["country_of_university"].notna()]["country_of_university"].item()

# Atribui os valores
df.loc[df["name_of_university"] == "Howard Hughes Medical Institute", ("city_of_university")] = city
df.loc[df["name_of_university"] == "Howard Hughes Medical Institute", ("country_of_university")] = country

# Imprime as instâncias com a universidade em questão
df[df["name_of_university"] == "Howard Hughes Medical Institute"]

,name,born_country_code,gender,year,category,share,name_of_university,city_of_university,country_of_university,age_get_prize
834,Robert J. Lefkowitz,US,male,2012,chemistry,2,Howard Hughes Medical Institute,Durham NC,USA,69
870,Paul Modrich,US,male,2015,chemistry,3,Howard Hughes Medical Institute,Durham NC,USA,69


Agora, o mesmo foi tentado com a LIGO/VIRGO Collaboration:

In [ ]:
# Imprime as instâncias com a universidade em questão
df[df["name_of_university"] == "LIGO/VIRGO Collaboration"]

,name,born_country_code,gender,year,category,share,name_of_university,city_of_university,country_of_university,age_get_prize
888,Rainer Weiss,DE,male,2017,physics,2,LIGO/VIRGO Collaboration,NaN,NaN,85
889,Barry C. Barish,US,male,2017,physics,4,LIGO/VIRGO Collaboration,NaN,NaN,81
890,Kip S. Thorne,US,male,2017,physics,4,LIGO/VIRGO Collaboration,NaN,NaN,77


Como todas as instâncias têm as mesmas ausências, elas não podem ser tratadas. Assim, caso seja feita alguma visualização em cima desses atributos, isso deve ser considerado.

Dessa forma, o resultado é:

In [ ]:
# Imprime a quantidade de valores NaN em cada atributo
count_nan(df)

Quantidade de valores NaN em:

name: 0
born_country_code: 0
gender: 0
year: 0
category: 0
share: 0
name_of_university: 226
city_of_university: 230
country_of_university: 229
age_get_prize: 0


###Relação entre cidade e país

Há também o caso em que as universidades possuem nome e país mas não cidade, ocorrendo isso com somente uma instituição, a University of Delaware:

In [ ]:
# Imprime as instâncias com a universidade em questão
df[df["name_of_university"] == "University of Delaware"]

,name,born_country_code,gender,year,category,share,name_of_university,city_of_university,country_of_university,age_get_prize
809,Richard F. Heck,US,male,2010,chemistry,3,University of Delaware,NaN,USA,79


Uma vez que todas as instâncias têm as mesmas ausências, elas novamente não podem ser tratadas. Assim, caso seja feita alguma visualização em cima desses atributos, isso também deve ser considerado.

Dessa forma, o resultado se mantém:

In [ ]:
# Imprime a quantidade de valores NaN em cada atributo
count_nan(df)

Quantidade de valores NaN em:

name: 0
born_country_code: 0
gender: 0
year: 0
category: 0
share: 0
name_of_university: 226
city_of_university: 230
country_of_university: 229
age_get_prize: 0


##Conversão do padrão do país de nascimento

Por fim no pré-processamento, o país de nascimento está no padrão `ISO 3166-1 alpha-2`, mas a biblioteca utiliza o `ISO 3166-1 alpha-3`. Dito isso, para gerar as visualizações, faz-se necessária essa conversão:

In [ ]:
# Converte de um padrão para o outro
convert_ISO_3166_2_to_3 = {"AF":"AFG", "AX":"ALA", "AL":"ALB", "DZ":"DZA", "AS":"ASM", "AD":"AND", "AO":"AGO", "AI":"AIA", "AQ":"ATA", "AG":"ATG", "AR":"ARG", "AM":"ARM", "AW":"ABW", "AU":"AUS", "AT":"AUT", "AZ":"AZE", "BS":"BHS", "BH":"BHR", "BD":"BGD", "BB":"BRB", "BY":"BLR", "BE":"BEL", "BZ":"BLZ", "BJ":"BEN", "BM":"BMU", "BT":"BTN", "BO":"BOL", "BA":"BIH", "BW":"BWA", "BV":"BVT", "BR":"BRA", "IO":"IOT", "BN":"BRN", "BG":"BGR", "BF":"BFA", "BI":"BDI", "KH":"KHM", "CM":"CMR", "CA":"CAN", "CV":"CPV", "KY":"CYM", "CF":"CAF", "TD":"TCD", "CL":"CHL", "CN":"CHN", "CX":"CXR", "CC":"CCK", "CO":"COL", "KM":"COM", "CG":"COG", "CD":"COD", "CK":"COK", "CR":"CRI", "CI":"CIV", "HR":"HRV", "CU":"CUB", "CY":"CYP", "CZ":"CZE", "DK":"DNK", "DJ":"DJI", "DM":"DMA", "DO":"DOM", "EC":"ECU", "EG":"EGY", "SV":"SLV", "GQ":"GNQ", "ER":"ERI", "EE":"EST", "ET":"ETH", "FK":"FLK", "FO":"FRO", "FJ":"FJI", "FI":"FIN", "FR":"FRA", "GF":"GUF", "PF":"PYF", "TF":"ATF", "GA":"GAB", "GM":"GMB", "GE":"GEO", "DE":"DEU", "GH":"GHA", "GI":"GIB", "GR":"GRC", "GL":"GRL", "GD":"GRD", "GP":"GLP", "GU":"GUM", "GT":"GTM", "GG":"GGY", "GN":"GIN", "GW":"GNB", "GY":"GUY", "HT":"HTI", "HM":"HMD", "VA":"VAT", "HN":"HND", "HK":"HKG", "HU":"HUN", "IS":"ISL", "IN":"IND", "ID":"IDN", "IR":"IRN", "IQ":"IRQ", "IE":"IRL", "IM":"IMN", "IL":"ISR", "IT":"ITA", "JM":"JAM", "JP":"JPN", "JE":"JEY", "JO":"JOR", "KZ":"KAZ", "KE":"KEN", "KI":"KIR", "KP":"PRK", "KR":"KOR", "KW":"KWT", "KG":"KGZ", "LA":"LAO", "LV":"LVA", "LB":"LBN", "LS":"LSO", "LR":"LBR", "LY":"LBY", "LI":"LIE", "LT":"LTU", "LU":"LUX", "MO":"MAC", "MK":"MKD", "MG":"MDG", "MW":"MWI", "MY":"MYS", "MV":"MDV", "ML":"MLI", "MT":"MLT", "MH":"MHL", "MQ":"MTQ", "MR":"MRT", "MU":"MUS", "YT":"MYT", "MX":"MEX", "FM":"FSM", "MD":"MDA", "MC":"MCO", "MN":"MNG", "ME":"MNE", "MS":"MSR", "MA":"MAR", "MZ":"MOZ", "MM":"MMR", "NA":"NAM", "NR":"NRU", "NP":"NPL", "NL":"NLD", "AN":"ANT", "NC":"NCL", "NZ":"NZL", "NI":"NIC", "NE":"NER", "NG":"NGA", "NU":"NIU", "NF":"NFK", "MP":"MNP", "NO":"NOR", "OM":"OMN", "PK":"PAK", "PW":"PLW", "PS":"PSE", "PA":"PAN", "PG":"PNG", "PY":"PRY", "PE":"PER", "PH":"PHL", "PN":"PCN", "PL":"POL", "PT":"PRT", "PR":"PRI", "QA":"QAT", "RE":"REU", "RO":"ROU", "RU":"RUS", "RW":"RWA", "BL":"BLM", "SH":"SHN", "KN":"KNA", "LC":"LCA", "MF":"MAF", "PM":"SPM", "VC":"VCT", "WS":"WSM", "SM":"SMR", "ST":"STP", "SA":"SAU", "SN":"SEN", "RS":"SRB", "SC":"SYC", "SL":"SLE", "SG":"SGP", "SK":"SVK", "SI":"SVN", "SB":"SLB", "SO":"SOM", "ZA":"ZAF", "GS":"SGS", "ES":"ESP", "LK":"LKA", "SD":"SDN", "SR":"SUR", "SJ":"SJM", "SZ":"SWZ", "SE":"SWE", "CH":"CHE", "SY":"SYR", "TW":"TWN", "TJ":"TJK", "TZ":"TZA", "TH":"THA", "TL":"TLS", "TG":"TGO", "TK":"TKL", "TO":"TON", "TT":"TTO", "TN":"TUN", "TR":"TUR", "TM":"TKM", "TC":"TCA", "TV":"TUV", "UG":"UGA", "UA":"UKR", "AE":"ARE", "GB":"GBR", "US":"USA", "UM":"UMI", "UY":"URY", "UZ":"UZB", "VU":"VUT", "VE":"VEN", "VN":"VNM", "VG":"VGB", "VI":"VIR", "WF":"WLF", "EH":"ESH", "YE":"YEM", "ZM":"ZMB", "ZW":"ZWE"}
df["born_country_code"] = df["born_country_code"].map(convert_ISO_3166_2_to_3.get)
df.head(5)

,name,born_country_code,gender,year,category,share,name_of_university,city_of_university,country_of_university,age_get_prize
0,Wilhelm Conrad Röntgen,DEU,male,1901,physics,1,Munich University,Munich,Germany,56
1,Hendrik A. Lorentz,NLD,male,1902,physics,2,Leiden University,Leiden,the Netherlands,49
2,Pieter Zeeman,NLD,male,1902,physics,2,Amsterdam University,Amsterdam,the Netherlands,37
3,Henri Becquerel,FRA,male,1903,physics,2,École Polytechnique,Paris,France,51
4,Pierre Curie,FRA,male,1903,physics,4,École municipale de physique et de chimie indu...,Paris,France,44


#Visualização

Tendo o dataset pronto, é viável fazer a visualização propriamente dita. Para isso, foi usada a biblioteca de gráficos interativos `plotly` para criar gráficos e gerar conclusões diversas dos dados.

##Comparação entre gêneros

Uma possível visualização seria alguma que comparasse ambos os gêneros presentes. Dessa maneira, decidiu-se primeiro analisar a distribuição entre homens e mulheres vencedores ao longo dos anos. Para isso, optou-se pelo *line plot*, pois é mais fácil de visualizar a diferença entre os gêneros em cada ponto do eixo x, e cores socialmente mais atribuídas a cada um:

In [ ]:
# Define a ordem dos gêneros para que as cores não mudem entre as visualizações
CATEGORY_ORDERS = {
    "gender": ["male", "female"]
}

In [ ]:
# Gera a visualização
fd = df.copy()
fd = fd.groupby(["year", "gender"]).agg("size").unstack().fillna(0)
px.line(fd, category_orders=CATEGORY_ORDERS, title="Vencedores do Nobel ao longo dos anos").show()

Assim, fica evidente que a diferença entre o número absoluto de homens e mulheres vencedores é considerável, sendo majoritariamente dominado por homens. Entretanto, percebe-se que após os anos 2000 houve um aumento na quantidade de mulheres, mesmo que tenha existido também um crescimento de premiados no geral. Por fim, um dado que merece destaque é o fato de não haver ganhadores de 1940 a 1942, devido à Segunda Guerra Mundial.

Em seguida, é interessante discriminar as categorias, a fim de identificar se a proporção observada se mantém entre elas. Continuando com as mesmas cores, optou-se pelo *grouped barplot*, pois é possível comparar tanto os gêneros entre si quanto em referência às categorias:

In [ ]:
# Gera a visualização
px.histogram(
    df,
    x="category",
    color="gender",
    barmode="group",
    title = "Vencedores do Nobel por categoria",
    category_orders=CATEGORY_ORDERS
).show()

Analisando a distribuição entre as áreas, nota-se que há mais mulheres em paz, medicina e literatura, enquanto existem apenas 10 participações em 100 anos em física, quimica e economia. Isso salienta como tais campos ainda são predominantemente masculinos e reforça a importância de políticas de incentivos para a participação feminina neles.

##Comparação entre compartilhamentos

A quantidade de autores por prêmio (se houve um ou mais ganhadores) foi outra visualização feita sobre as caregorias. Optou-se pelo *stacked barplot* pois fica claro como os possíveis valores compõem um todo, o que possibilita compará-los dentro de uma mesma categoria, além de entre elas, sendo que as cores escolhidas são arbitrárias e sem signficado, mas com o cuidado de não repetir as tonalidades já usadas:

In [ ]:
# Ordena crescentemente os compartilhamentos
dfaux = df
b, c = df.iloc[920].copy(), df.iloc[4].copy()
df.iloc[920], df.iloc[4] = c, b

# Gera a visualização
px.histogram(
    dfaux,
    x="category",
    color="share",
    title = "Compartilhamento do Nobel por categoria",
    color_discrete_sequence=[px.colors.qualitative.Set1[i] for i in [0, 1, 2, 5]]
).update_xaxes(categoryorder="total descending").update_layout(legend_traceorder="normal").show()

Percebe-se que medicina, física e química possuem a maior quantidade de compartilhamentos, além de economia também ser bem balanceada, provavelmente em decorrência da complexidade de seus feitos. Por outro lado, literatura tem 108 trabalhos individuais e apenas 8 em duplas, o que evidencia um caráter mais pessoal da categoria e de suas obras, assim como paz.

##Comparação entre ganhadores

Em seguida, decidimos analisar se havia alguma pessoa que havia recebido o prêmio mais de uma vez:

**[Por que do gráfico e das cores?]**

In [ ]:
# 
def frequency(df, new_col: str, freq_col: str, cols=None):
    if cols is None:
        cols = []

    fd = df[[freq_col] + cols].copy()
    fd[new_col] = fd.groupby(freq_col)[freq_col].transform("size")

    return fd.drop_duplicates(freq_col)

In [ ]:
# Gera a visualização
fd = frequency(df, "count", "name", ["gender", "born_country_code"])
fd = frequency(fd, "value", "count")
fd["count"] = fd["count"].map(lambda x: f"Pessoas com {x} prêmio" if x == 1 else f"Pessoas com {x} prêmios")
px.pie(
    fd,
    values="value",
    names="count",
    title="Quantidade de Nobel por pessoa",
    color_discrete_sequence=(px.colors.qualitative.Plotly[3:])
).show()

Contando a frequência dos dados percebemos que 4 pessoas receberam o prêmio 2x, sendo elas Marie Curie, John Bardeen, Linus Pauling e Frederick Sanger. O que resulta numa fatia muito pequena de 0.4% das pessoas que possuem 2 Nobels

##Comparação entre países

Considerando o dado do país em que nasceu, fizemos outro histograma para visualizar a diferença entre as nações:

**[Por que do gráfico e das cores?]**

In [ ]:
# Gera a visualização
fig = px.histogram(df, x="born_country_code", title="Vencedores do Nobel por país")
fig.update_xaxes(categoryorder="total descending")
fig.show()

Interagindo com o gráfico, é possivel ver que os três primeiros países no ranking já totalizam quase 50% dos prêmios, sendo os Estados Unidos o país com a maior quantidade por uma larga vantagem.
Selecionando os países menores é possivel perceber que a grande maioria não chega aos 20 prêmios.

Tal informação pode ter duas iterpretações: ou os Estados Unidos, Reino Unido e Alemanha estão cientificamente muito a frente dos outros países, ou ao longo dos anos esses países foram privilegiádos por escolhas não tão imparciais dos organizadores do prêmio.

Complementando tal vizualização, podemos criar uma escala de calor sobre o próprio mapa do mundo, para vizualizar a ocorrencia dos premios distribuídos no mundo:

**[Por que do gráfico e das cores?]**

In [ ]:
# Gera a visualização
fd = frequency(df, "count", "born_country_code")
fig = go.Figure(data=go.Choropleth(
    locations=fd["born_country_code"],
    z=fd["count"],
    colorscale="burg",
    #title="Vencedores do Nobel por país"
))
fig.show()

Texto

**[Por que do gráfico e das cores?]**

In [ ]:
# 
country_to_continent = {"AFG": "Asia", "ALB": "Europe", "ATA": "Antarctica", "DZA": "Africa", "ASM": "Oceania", "AND": "Europe", "AGO": "Africa", "ATG": "North America", "AZE": "Europe", "AZE": "Asia", "ARG": "South America", "AUS": "Oceania", "AUT": "Europe", "BHS": "North America", "BHR": "Asia", "BGD": "Asia", "ARM": "Europe", "ARM": "Asia", "BRB": "North America", "BEL": "Europe", "BMU": "North America", "BTN": "Asia", "BOL": "South America", "BIH": "Europe", "BWA": "Africa", "BVT": "Antarctica", "BRA": "South America", "BLZ": "North America", "IOT": "Asia", "SLB": "Oceania", "VGB": "North America", "BRN": "Asia", "BGR": "Europe", "MMR": "Asia", "BDI": "Africa", "BLR": "Europe", "KHM": "Asia", "CMR": "Africa", "CAN": "North America", "CPV": "Africa", "CYM": "North America", "CAF": "Africa", "LKA": "Asia", "TCD": "Africa", "CHL": "South America", "CHN": "Asia", "TWN": "Asia", "CXR": "Asia", "CCK": "Asia", "COL": "South America", "COM": "Africa", "MYT": "Africa", "COG": "Africa", "COD": "Africa", "COK": "Oceania", "CRI": "North America", "HRV": "Europe", "CUB": "North America", "CYP": "Europe", "CYP": "Asia", "CZE": "Europe", "BEN": "Africa", "DNK": "Europe", "DMA": "North America", "DOM": "North America", "ECU": "South America", "SLV": "North America", "GNQ": "Africa", "ETH": "Africa", "ERI": "Africa", "EST": "Europe", "FRO": "Europe", "FLK": "South America", "SGS": "Antarctica", "FJI": "Oceania", "FIN": "Europe", "ALA": "Europe", "FRA": "Europe", "GUF": "South America", "PYF": "Oceania", "ATF": "Antarctica", "DJI": "Africa", "GAB": "Africa", "GEO": "Europe", "GEO": "Asia", "GMB": "Africa", "PSE": "Asia", "DEU": "Europe", "GHA": "Africa", "GIB": "Europe", "KIR": "Oceania", "GRC": "Europe", "GRL": "North America", "GRD": "North America", "GLP": "North America", "GUM": "Oceania", "GTM": "North America", "GIN": "Africa", "GUY": "South America", "HTI": "North America", "HMD": "Antarctica", "VAT": "Europe", "HND": "North America", "HKG": "Asia", "HUN": "Europe", "ISL": "Europe", "IND": "Asia", "IDN": "Asia", "IRN": "Asia", "IRQ": "Asia", "IRL": "Europe", "ISR": "Asia", "ITA": "Europe", "CIV": "Africa", "JAM": "North America", "JPN": "Asia", "KAZ": "Europe", "KAZ": "Asia", "JOR": "Asia", "KEN": "Africa", "PRK": "Asia", "KOR": "Asia", "KWT": "Asia", "KGZ": "Asia", "LAO": "Asia", "LBN": "Asia", "LSO": "Africa", "LVA": "Europe", "LBR": "Africa", "LBY": "Africa", "LIE": "Europe", "LTU": "Europe", "LUX": "Europe", "MAC": "Asia", "MDG": "Africa", "MWI": "Africa", "MYS": "Asia", "MDV": "Asia", "MLI": "Africa", "MLT": "Europe", "MTQ": "North America", "MRT": "Africa", "MUS": "Africa", "MEX": "North America", "MCO": "Europe", "MNG": "Asia", "MDA": "Europe", "MNE": "Europe", "MSR": "North America", "MAR": "Africa", "MOZ": "Africa", "OMN": "Asia", "NAM": "Africa", "NRU": "Oceania", "NPL": "Asia", "NLD": "Europe", "ANT": "North America", "CUW": "North America", "ABW": "North America", "SXM": "North America", "BES": "North America", "NCL": "Oceania", "VUT": "Oceania", "NZL": "Oceania", "NIC": "North America", "NER": "Africa", "NGA": "Africa", "NIU": "Oceania", "NFK": "Oceania", "NOR": "Europe", "MNP": "Oceania", "UMI": "Oceania", "UMI": "North America", "FSM": "Oceania", "MHL": "Oceania", "PLW": "Oceania", "PAK": "Asia", "PAN": "North America", "PNG": "Oceania", "PRY": "South America", "PER": "South America", "PHL": "Asia", "PCN": "Oceania", "POL": "Europe", "PRT": "Europe", "GNB": "Africa", "TLS": "Asia", "PRI": "North America", "QAT": "Asia", "REU": "Africa", "ROU": "Europe", "RUS": "Europe", "RUS": "Asia", "RWA": "Africa", "BLM": "North America", "SHN": "Africa", "KNA": "North America", "AIA": "North America", "LCA": "North America", "MAF": "North America", "SPM": "North America", "VCT": "North America", "SMR": "Europe", "STP": "Africa", "SAU": "Asia", "SEN": "Africa", "SRB": "Europe", "SYC": "Africa", "SLE": "Africa", "SGP": "Asia", "SVK": "Europe", "VNM": "Asia", "SVN": "Europe", "SOM": "Africa", "ZAF": "Africa", "ZWE": "Africa", "ESP": "Europe", "SSD": "Africa", "ESH": "Africa", "SDN": "Africa", "SUR": "South America", "SJM": "Europe", "SWZ": "Africa", "SWE": "Europe", "CHE": "Europe", "SYR": "Asia", "TJK": "Asia", "THA": "Asia", "TGO": "Africa", "TKL": "Oceania", "TON": "Oceania", "TTO": "North America", "ARE": "Asia", "TUN": "Africa", "TUR": "Europe", "TUR": "Asia", "TKM": "Asia", "TCA": "North America", "TUV": "Oceania", "UGA": "Africa", "UKR": "Europe", "MKD": "Europe", "EGY": "Africa", "GBR": "Europe", "GGY": "Europe", "JEY": "Europe", "IMN": "Europe", "TZA": "Africa", "USA": "North America", "VIR": "North America", "BFA": "Africa", "URY": "South America", "UZB": "Asia", "VEN": "South America", "WLF": "Oceania", "WSM": "Oceania", "YEM": "Asia", "ZMB": "Africa"}

# Gera a visualização
fd = df.copy()
fd["born_continent"] = fd["born_country_code"].map(country_to_continent.get)
px.treemap(fd, path=[px.Constant("World"), "born_continent", "born_country_code"], title="Vencedores do Nobel por continente").show()

Texto

#Conclusão

Após processar e analisar os dados por diferentes vias e utilizando diferentes técnicas foi possivel perceber que todas elas tem um papel importante na compreensão dos dados. Alguns gráficos trazem de maneira mais fácil e visual dados que seriam muito difíceis de perceber com métricas e mineiração de addos, enquanto o uso de métricas e avaliação de nulos pode ser, inclusive, mais eficiente para a obtenção de informação.

A atividade prática nos permitiu entrar em contato com uma situação real de ciência de dados e mostrou que nem sempre é fácil transformar dados e informação e entender dados vazios ou não muito relevantes. 

# DashBoard


In [ ]:
from ipywidgets import widgets


wid_query = {
    "query": widgets.Text(
        value="",
        placeholder="Type a year to see the Winners",
        description="Query: ",
        disabled=False),
    "button": widgets.Button(
        description='Search',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        icon='search', # (FontAwesome names without the `fa-` prefix)
        tooltip='Search for a country'),
    "output": widgets.Label(value="")}

def on_query(button):
    wid_query["output"].value = df[df['year'].astype(str) == wid_query["query"].value]['name'].to_string(index=False)
    
# Eventos
wid_query["button"].on_click(on_query)

#slider
years = df["year"].tolist()

wid_years = {
    "play": widgets.Play(
        value=years[0],
        min=years[0],
        max=years[-1],
        step=1,
        interval=500,
        description="Years",
        disabled=False),
    "slider": widgets.IntSlider(
        value=years[0],
        min=years[0],
        max=years[-1],
        step=1)}

widgets.jslink((wid_years["play"], 'value'),
               (wid_years["slider"], 'value'))

widgets.VBox([
    widgets.HBox([wid_query["query"], wid_query["button"]]),
    wid_query["output"],
    wid_years["play"], wid_years["slider"]]
    )